In [1]:
from bs4 import BeautifulSoup
import requests
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

# URl to web scrap from.
page_url = "https://www.amazon.com/All-New-Insignia-NS-65DF710NA21-65-inch-Smart/dp/B08CVV2Z32/"


webpage = requests.get(page_url, headers=HEADERS)

soup = BeautifulSoup(webpage.content, "lxml")

# Outer Tag Object and attributes
title = soup.find("span", attrs={"id":'productTitle'}).string.strip()
# This is a special deal, so the id is named dealprice and not ourprice
#price = soup.find("span", attrs={"id":'priceblock_ourprice'}).string.strip()
price = soup.find("span", attrs={"id":'priceblock_dealprice'}).string.strip()
rating = soup.find("i", attrs={"class":'a-icon a-icon-star a-star-4-5'}).string.strip()
size = soup.find("span", attrs={"class":'selection'}).string.strip()

# The product is not available for Germany, so no shipping price
#shipping = soup.find("span", attrs={"class":"a-size-base a-color-secondary"}).string.strip()

In [2]:
title

'All New Insignia NS-70DF710NA21 70-inch Smart 4K UHD - FireTV Edition, Released 2020'

In [3]:
rating

'4.6 out of 5 stars'

In [4]:
size

'70-inch'

In [5]:
price

'$549.99'

In [6]:
# Put everything intoSQL db
DB_NAME = 'amazon'


cnx = mysql.connector.connect(user='root', password='kieselgur')
cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

cursor.close()
cnx.close()

In [13]:
cnx = mysql.connector.connect(user='root', password='kieselgur', database=DB_NAME)
cursor = cnx.cursor()

TABLES = {}
TABLES['televisionsets'] = (
    "CREATE TABLE `televisionsets` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `title` varchar(200) NOT NULL,"
    "  `rating` varchar(64) NOT NULL,"
    "  `size` varchar(64) NOT NULL,"
    "  `price` varchar(64) NOT NULL,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table televisionsets: OK


In [14]:
cnx = mysql.connector.connect(user='root', password='kieselgur', database=DB_NAME)
cursor = cnx.cursor()

add_tv = ("INSERT INTO televisionsets "
              "(title, rating, size, price) "
              "VALUES (%s, %s, %s, %s)")

data_tv = ( title, rating, size, price )

cursor.execute(add_tv, data_tv)
#id = cursor.lastrowid

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()
